# Tutorial 1 - Regression

We will predict the price (`price` column) of an AirBNB listing in Boston given a number of features about the listing.

**Therefore, our unit of analysis is an AIRBNB LISTING**

# Setup

In [26]:
# Common imports
import numpy as np
import pandas as pd

np.random.seed(42)


# Get the data

In [27]:
#We will predict the "price" value in the data set:

airbnb = pd.read_csv("airbnb.csv")
airbnb.head()

,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,...,guests_included,price_per_extra_person,minimum_nights,number_of_reviews,number_days_btw_first_last_review,review_scores_rating,cancellation_policy,price,price_gte_150,price_category
0,0,0,Roslindale,42.282619,-71.133068,House,Entire home/apt,4,1.5,2.0,...,1,0,2,0,0,NaN,moderate,250,1,gte_226
1,0,1,Roslindale,42.286241,-71.134374,Apartment,Private room,2,1.0,1.0,...,0,0,2,36,804,94.0,moderate,65,0,lte_75
2,1,1,Roslindale,42.292438,-71.135765,Apartment,Private room,2,1.0,1.0,...,1,20,3,41,2574,98.0,moderate,65,0,lte_75
3,0,0,Roslindale,42.281106,-71.121021,House,Private room,4,1.0,1.0,...,2,25,1,1,0,100.0,moderate,75,0,lte_75
4,1,1,Roslindale,42.284512,-71.136258,House,Private room,2,1.5,1.0,...,1,0,2,29,380,99.0,flexible,79,0,btw_75-150


# Split the data into train and test

In [30]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(airbnb, test_size=0.3)

### Be careful: we haven't seperated the target column yet

## Check the missing values

In [31]:
    train_set.isna().sum()

host_is_superhost                       0
host_identity_verified                  0
neighbourhood_cleansed                  0
latitude                                0
longitude                               0
property_type                           6
room_type                               0
accommodates                            0
bathrooms                              26
bedrooms                               19
beds                                   21
bed_type                                0
Number of amenities                     0
guests_included                         0
price_per_extra_person                  0
minimum_nights                          0
number_of_reviews                       0
number_days_btw_first_last_review       0
review_scores_rating                 1560
cancellation_policy                     0
price                                   0
price_gte_150                           0
price_category                          0
dtype: int64

In [32]:
test_set.isna().sum()

host_is_superhost                      0
host_identity_verified                 0
neighbourhood_cleansed                 0
latitude                               0
longitude                              0
property_type                          3
room_type                              0
accommodates                           0
bathrooms                             10
bedrooms                              11
beds                                   3
bed_type                               0
Number of amenities                    0
guests_included                        0
price_per_extra_person                 0
minimum_nights                         0
number_of_reviews                      0
number_days_btw_first_last_review      0
review_scores_rating                 723
cancellation_policy                    0
price                                  0
price_gte_150                          0
price_category                         0
dtype: int64

# Data Prep

In [33]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

## Drop the variables we can't use in this tutorial

In [34]:
# We can't use the following columns in this tutorial, because they are for classification tasks

train = train_set.drop(['price_gte_150', 'price_category'], axis=1)
test = test_set.drop(['price_gte_150', 'price_category'], axis=1)

## Separate the target variable (we don't want to transform it)

In [35]:
train_y = train[['price']]
test_y = test[['price']]

train_inputs = train.drop(['price'], axis=1)
test_inputs = test.drop(['price'], axis=1)

##  Identify the numerical and categorical columns

In [36]:
train_inputs.dtypes

host_is_superhost                      int64
host_identity_verified                 int64
neighbourhood_cleansed                object
latitude                             float64
longitude                            float64
property_type                         object
room_type                             object
accommodates                           int64
bathrooms                            float64
bedrooms                             float64
beds                                 float64
bed_type                              object
Number of amenities                    int64
guests_included                        int64
price_per_extra_person                 int64
minimum_nights                         int64
number_of_reviews                      int64
number_days_btw_first_last_review      int64
review_scores_rating                 float64
cancellation_policy                   object
dtype: object

**At this stage, you can manually identify numeric, binary, and categorical columns as follows:**

`numeric_columns = ['latitude', 'longitude', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'Number of amenities', 'guests_included', 'price_per_extra_person', 'minimum_nights', 'number_of_reviews', 'number_days_btw_first_last_review', 'review_scores_rating']`
 
 `binary_columns = ['host_is_superhost', 'host_identity_verified']`
 
 `categorical_columns = ['neighbourhood_cleansed', 'property_type', 'room_type', 'bed_type', 'cancellation_policy']`
 
<br>
 
**If you do not want to manually type these, you can do the below tricks:**

In [68]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [69]:
# Identify the binary columns so we can pass them through without transforming
binary_columns = ['host_is_superhost', 'host_identity_verified']

In [70]:
# Be careful: numerical columns already includes the binary columns,
# So, we need to remove the binary columns from numerical columns.

for col in binary_columns:
    numeric_columns.remove(col)
# numeric_columns = [col for col in numeric_columns if col not in binary_columns]
# numeric_columns

In [71]:
binary_columns

['host_is_superhost', 'host_identity_verified']

In [72]:
numeric_columns

['latitude',
 'longitude',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'Number of amenities',
 'guests_included',
 'price_per_extra_person',
 'minimum_nights',
 'number_of_reviews',
 'number_days_btw_first_last_review',
 'review_scores_rating']

In [74]:
categorical_columns

['neighbourhood_cleansed',
 'property_type',
 'room_type',
 'bed_type',
 'cancellation_policy']

# Pipeline

In [75]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='mean')),
                ('scaler', StandardScaler())])

In [76]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [77]:
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])

In [78]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('binary', binary_transformer, binary_columns)],
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for TRAIN

In [79]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

array([[ 0.77237875,  0.49233717,  0.58858218, ...,  0.        ,
         1.        ,  1.        ],
       [ 0.97659977,  0.95357556, -0.57787478, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.76426124, -1.48371607,  0.58858218, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.8520649 ,  0.66551477, -0.57787478, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.04051318,  1.10075113,  1.75503913, ...,  0.        ,
         0.        ,  1.        ],
       [-0.23269918, -0.68229914, -0.57787478, ...,  0.        ,
         0.        ,  1.        ]])

In [80]:
train_x.shape

(7190, 61)

# Tranform: transform() for TEST

In [81]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

array([[ 0.14120446, -0.49561407,  1.17181065, ...,  0.        ,
         0.        ,  1.        ],
       [-1.08251342, -1.01992797,  0.58858218, ...,  0.        ,
         1.        ,  1.        ],
       [ 0.26172187, -0.27562275, -0.57787478, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.27224811,  0.21644787, -1.16110326, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.58301998,  0.75400318,  0.58858218, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.08505909,  1.04710196,  1.75503913, ...,  0.        ,
         0.        ,  0.        ]])

In [82]:
test_x.shape

(3082, 61)

# Train a Linear Regression model

In [83]:
#Closed form solution

from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()

lin_reg.fit(train_x, train_y)

LinearRegression()

In [84]:
from sklearn.metrics import mean_squared_error

In [85]:
#Train RMSE
reg_train_pred = lin_reg.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 52.23472864746545


In [86]:
#Test RMSE
reg_test_pred = lin_reg.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 53.09101178161191


# Calculate the baseline

In [87]:
#First find the average value of the target

mean_value = np.mean(train_y['price'])

mean_value

160.5344923504868

In [88]:
# Predict all values as the mean

baseline_pred = np.repeat(mean_value, len(test_y))

baseline_pred

array([160.53449235, 160.53449235, 160.53449235, ..., 160.53449235,
       160.53449235, 160.53449235])

In [89]:
baseline_mse = mean_squared_error(test_y, baseline_pred)

baseline_rmse = np.sqrt(baseline_mse)

print('Baseline RMSE: {}' .format(baseline_rmse))

Baseline RMSE: 89.90931562531277


In [90]:
train_y['price']

8754    300
7456    150
3173    195
1464    249
3393     69
       ... 
6437    190
7394    219
5241    157
9950    250
7338    100
Name: price, Length: 7190, dtype: int64

# Train a Stochastic Gradient Regression Model

In [91]:
from sklearn.linear_model import SGDRegressor 

# tol = stopping criterion
# eta0 = learning rate
# penalty = regularization term
# max_iter = number of passes over training data (i.e., epochs)

sgd_reg = SGDRegressor(max_iter=100, penalty=None, eta0=0.1, tol=0.0001) 

sgd_reg.fit(train_x, train_y)

c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(eta0=0.1, max_iter=100, penalty=None, tol=0.0001)

In [92]:
sgd_reg.n_iter_

16

In [93]:
#Train RMSE
reg_train_pred = sgd_reg.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 53.510405245342994


In [94]:
#Test RMSE
reg_test_pred = sgd_reg.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 54.17685435654117


# Polynomial Regression

This is done by creating the polynomial "variables" of the existing variables, then fitting them in a regular regression model


In [95]:
from sklearn.preprocessing import PolynomialFeatures

# Create second degree terms and interaction terms
poly_features = PolynomialFeatures(degree=2).fit(train_x)

train_x_poly = poly_features.transform(train_x)

test_x_poly = poly_features.transform(test_x)

#Mind you, this will create the polynomial terms of the categorical variables too

#if degree=3, then it creates all combinations: a, a^2, a^3, b, b^2, b^3, a.b, a^2.b, a.b^2, a^2.b^2 

In [96]:
#We still fit a linear regression model

pol_lin_reg = LinearRegression()

pol_lin_reg.fit(train_x_poly, train_y)

LinearRegression()

In [97]:
#Train RMSE
reg_train_pred = pol_lin_reg.predict(train_x_poly)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 39.15795003511119


In [98]:
#Test RMSE
reg_test_pred = pol_lin_reg.predict(test_x_poly)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 10830183581.352474


## L2 Regularization

In [99]:
# Closed form solution:
# Remember, alpha is the magnitude of regularization

from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha=0.1, solver='cholesky')

ridge_reg.fit(train_x, train_y)

Ridge(alpha=0.1, solver='cholesky')

In [100]:
#Train RMSE
reg_train_pred = ridge_reg.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 52.23238950327906


In [101]:
#Test RMSE
reg_test_pred = ridge_reg.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 53.08703238125635


### Stochastic Gradient Descent version:

In [102]:
#Stochastic Gradient:

sgd_reg_L2 = SGDRegressor(max_iter=50, penalty='l2', alpha = 0.1, eta0=0.1, tol=0.0001)

sgd_reg_L2.fit(train_x, train_y)

c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(alpha=0.1, eta0=0.1, max_iter=50, tol=0.0001)

In [103]:
#Train RMSE
reg_train_pred = sgd_reg_L2.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 58.13125217156614


In [104]:
#Test RMSE
reg_test_pred = sgd_reg_L2.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 58.42795075161912


## L1 Regularization

In [105]:
# Closed form solution:
# Remember, alpha is the magnitude of regularization

from sklearn.linear_model import Lasso

lasso_reg = Lasso(alpha=0.1, max_iter=100000)

lasso_reg.fit(train_x, train_y)

Lasso(alpha=0.1, max_iter=100000)

In [106]:
#Train RMSE
reg_train_pred = lasso_reg.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 52.446522338363295


In [107]:
#Test RMSE
reg_test_pred = lasso_reg.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 53.21235944947543


### Stochastic Gradient Descent version:

In [108]:
#Stochastic Gradient:
sgd_reg_L1 = SGDRegressor(max_iter=50, penalty='l1', alpha = 0.1, eta0=0.1, tol=0.0001)

sgd_reg_L1.fit(train_x, train_y)

c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(alpha=0.1, eta0=0.1, max_iter=50, penalty='l1', tol=0.0001)

In [109]:
#Train RMSE
reg_train_pred = sgd_reg_L1.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 52.68942418921106


In [110]:
#Test RMSE
reg_test_pred = sgd_reg_L1.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 53.479571358906334


## ElasticNet

In [111]:
# Remember, alpha is the magnitude of regularization
# And, l1_ratio is how much L1 vs. L2 regularization to do.

from sklearn.linear_model import ElasticNet

elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5)

elastic_net.fit(train_x, train_y)

ElasticNet(alpha=0.1)

In [112]:
#Train RMSE
reg_train_pred = elastic_net.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 54.60136747463492


In [113]:
#Test RMSE
reg_test_pred = elastic_net.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 54.919824127471216


### Stochastic Gradient Descent version:

In [114]:
#Stochastic Gradient:
sgd_reg_elastic = SGDRegressor(max_iter=50, penalty='elasticnet', l1_ratio=0.5, alpha = 0.1, 
                          eta0=0.1, tol=0.0001)
sgd_reg_elastic.fit(train_x, train_y)

c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(alpha=0.1, eta0=0.1, l1_ratio=0.5, max_iter=50,
             penalty='elasticnet', tol=0.0001)

In [115]:
#Train RMSE
reg_train_pred = sgd_reg_elastic.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 56.27611287078275


In [116]:
#Test RMSE
reg_test_pred = sgd_reg_elastic.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 56.36512342875374


## Early Stopping

In [117]:
np.random.seed(42)

sgd_reg_ES = SGDRegressor(max_iter=500, early_stopping = True, n_iter_no_change=5, 
                          validation_fraction=0.2,
                          eta0=0.01, tol=0.0001)

In [118]:
sgd_reg_ES.fit(train_x, train_y)

c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(early_stopping=True, max_iter=500, tol=0.0001,
             validation_fraction=0.2)

### Number of iterations with early stopping

In [119]:
sgd_reg_ES.n_iter_

12

In [120]:
#Train RMSE
reg_train_pred = sgd_reg_ES.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 52.738448409249514


In [121]:
#Test RMSE
reg_test_pred = sgd_reg_ES.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 53.565363527454785


### Number of iterations without early stopping

In [122]:
np.random.seed(42)

sgd_reg_ES = SGDRegressor(max_iter=500, eta0=0.01, tol=0.0001)

In [123]:
sgd_reg_ES.fit(train_x, train_y)

c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(max_iter=500, tol=0.0001)

In [124]:
sgd_reg_ES.n_iter_

34

In [125]:
#Train RMSE
reg_train_pred = sgd_reg_ES.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 52.39455160089534


In [126]:
#Test RMSE
reg_test_pred = sgd_reg_ES.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 53.22337977552451


## Remember: Polynomial model has severe overfitting. Perfom regualization on it?

## ElasticNet

In [127]:
# Remember, alpha is the magnitude of regularization
# And, l1_ratio is how much L1 vs. L2 regularization to do.

from sklearn.linear_model import ElasticNet

elastic_net = ElasticNet(alpha=0.05, l1_ratio=0.5)

elastic_net.fit(train_x_poly, train_y)

c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.224e+04, tolerance: 5.978e+03
  model = cd_fast.enet_coordinate_descent(


ElasticNet(alpha=0.05)

In [128]:
#Train RMSE
reg_train_pred = elastic_net.predict(train_x_poly)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 45.311311507126376


In [129]:
#Test RMSE
reg_test_pred = elastic_net.predict(test_x_poly)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 48.15178607905508
